<a href="https://colab.research.google.com/github/Artyom995/Parsing_OKX/blob/main/Parsing_OKX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium numpy keras tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 14.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import time
import numpy as np
import pandas as pd
from collections import deque
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib
from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class MarketAnalyzer:
    def __init__(self, symbol):
        """Инициализация анализатора рынка"""
        self.symbol = symbol
        self.SEQUENCE_LENGTH = 10
        self.ORDER_LIMIT = 13
        self.BUFFER_SIZE = 1000  # размер буфера
        self.RETRAIN_INTERVAL = 300  # 5 минут в секундах
        self.INITIAL_TRAIN_SIZE = 200  # размер начального обучения

        # Список признаков
        self.FEATURES = [
            'ask_price', 'ask_amount', 'ask_total',
            'bid_price', 'bid_amount', 'bid_total',
            'imbalance', 'absorption_ratio', 'depth_pressure',
            'volatility'
        ]

        # Инициализация модели
        try:
            self.model = load_model(f'{symbol}_model.h5')
            self.scaler = joblib.load(f'{symbol}_scaler.pkl')
        except:
            self._initialize_new_model()

        # Инициализация буферов
        self.data_buffer = deque(maxlen=self.BUFFER_SIZE)
        self.data_sequence = deque(maxlen=self.SEQUENCE_LENGTH)
        self.last_retrain_time = time.time()
        self.price_history = deque(maxlen=50)
        self.initial_training_done = False
        self.model_accuracy = 0.0  # Точность модели

        # Настройка Selenium
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--window-size=1800,1000")
        self.driver = webdriver.Chrome(options=options)
        self.driver.set_window_size(1800, 1000)
        self.driver.get("https://www.okx.com/ru/trade-swap/sol-usdt-swap")

    def _initialize_new_model(self):
        """Инициализация новой модели"""
        self.model = Sequential([
            LSTM(32, input_shape=(self.SEQUENCE_LENGTH, len(self.FEATURES))),
            Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.scaler = MinMaxScaler()

    def parse_number(self, s):
        """Преобразование строки в float, учитывая суффиксы и запятые"""
        s = s.replace(',', '.')
        if s.endswith('K'):
            return float(s[:-1]) * 1000
        elif s.endswith('M'):
            return float(s[:-1]) * 1000000
        else:
            return float(s)

    async def fetch_order_book(self):
        """Асинхронное получение данных книги ордеров и текущей цены с помощью Selenium"""
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.index_asks__UjOQB, li.index_bids__0-C60"))
            )
            asks_elements = self.driver.find_elements(By.CSS_SELECTOR, "li.index_asks__UjOQB")[:self.ORDER_LIMIT]
            bids_elements = self.driver.find_elements(By.CSS_SELECTOR, "li.index_bids__0-C60")[:self.ORDER_LIMIT]

            asks = []
            for elem in asks_elements:
                try:
                    price_str = elem.find_element(By.CSS_SELECTOR, 'button.index_price__xLB8Q').text
                    amount_str = elem.find_element(By.CSS_SELECTOR, 'span.index_amount__2RZYj').text
                    price = self.parse_number(price_str)
                    amount = self.parse_number(amount_str)
                    asks.append([price, amount])
                except Exception as e:
                    print(f"Ошибка парсинга ask: {e}")

            bids = []
            for elem in bids_elements:
                try:
                    price_str = elem.find_element(By.CSS_SELECTOR, 'button.index_price__xLB8Q').text
                    amount_str = elem.find_element(By.CSS_SELECTOR, 'span.index_amount__2RZYj').text
                    price = self.parse_number(price_str)
                    amount = self.parse_number(amount_str)
                    bids.append([price, amount])
                except Exception as e:
                    print(f"Ошибка парсинга bid: {e}")

            # Парсинг текущей цены
            current_price_elem = self.driver.find_element(By.CSS_SELECTOR, "button.okui-plain-button.index_tickerPrice__0ElT5")
            current_price_str = current_price_elem.text
            current_price = self.parse_number(current_price_str)

            return asks, bids, current_price
        except Exception as e:
            print(f"Ошибка получения данных: {e}")
            return None, None, None

    def calculate_absorption(self, asks, bids):
        """Расчет показателей поглощения ликвидности"""
        absorption_features = {}

        ask_levels = [(ask[0], ask[1]) for ask in asks]
        ask_volumes = [vol for price, vol in ask_levels]
        absorption_features['ask_liquidity'] = sum(ask_volumes)
        absorption_features['ask_absorption'] = sum(ask_volumes[:15])

        bid_levels = [(bid[0], bid[1]) for bid in bids]
        bid_volumes = [vol for price, vol in bid_levels]
        absorption_features['bid_liquidity'] = sum(bid_volumes)
        absorption_features['bid_absorption'] = sum(bid_volumes[:15])

        absorption_features['absorption_ratio'] = (
            absorption_features['bid_absorption'] - absorption_features['ask_absorption']
        ) / (absorption_features['bid_absorption'] + absorption_features['ask_absorption'] + 1e-8)

        absorption_features['depth_pressure'] = (
            sum(vol * (i+1) for i, vol in enumerate(bid_volumes)) -
            sum(vol * (i+1) for i, vol in enumerate(ask_volumes))
        ) / 1000

        return absorption_features

    def calculate_volatility(self):
        """Расчет волатильности на основе истории цен"""
        if len(self.price_history) < 2:
            return 0
        returns = np.diff(list(self.price_history)) / list(self.price_history)[:-1]
        return np.std(returns) * 100

    def process_data(self, asks, bids, current_price):
        """Обработка данных книги ордеров"""
        if not asks or not bids:
            return None

        absorption = self.calculate_absorption(asks, bids)
        self.price_history.append(current_price)

        data = {
            'ask_price': asks[0][0],
            'ask_amount': asks[0][1],
            'ask_total': sum(ask[0] * ask[1] for ask in asks),
            'bid_price': bids[0][0],
            'bid_amount': bids[0][1],
            'bid_total': sum(bid[0] * bid[1] for bid in bids),
            'imbalance': (sum(bid[1] for bid in bids) - sum(ask[1] for ask in asks)) /
                        (sum(bid[1] for bid in bids) + sum(ask[1] for ask in asks)),
            'volatility': self.calculate_volatility(),
            'current_price': current_price  # Для обучения модели
        }
        data.update(absorption)

        df = pd.DataFrame([data])
        self.scaler.partial_fit(df[self.FEATURES])  # Обновляем scaler для новых данных
        return df

    async def retrain_model(self):
        """Переобучение модели с использованием новых данных"""
        df = pd.DataFrame(list(self.data_buffer))
        X = df[self.FEATURES]
        y = (df['current_price'].shift(-5) > df['current_price']).astype(int).values  # Прогноз на 5 шагов вперед

        X_scaled = self.scaler.transform(X)  # Используем transform, а не fit_transform

        X_seq, y_seq = [], []
        for i in range(self.SEQUENCE_LENGTH, len(X_scaled) - 5):  # Учитываем сдвиг на 5 шагов
            X_seq.append(X_scaled[i-self.SEQUENCE_LENGTH:i])
            y_seq.append(y[i])

        if len(X_seq) > 0:
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
            history = self.model.fit(np.array(X_seq), np.array(y_seq),
                                   epochs=50, batch_size=32, validation_split=0.2,
                                   callbacks=[early_stopping], verbose=1)
            self.model_accuracy = history.history['val_accuracy'][-1] * 100
            self.model.save(f'{self.symbol}_model.h5')
            joblib.dump(self.scaler, f'{self.symbol}_scaler.pkl')
            print(f"Модель {self.symbol} успешно переобучена")

    def predict_movement(self, X_input):
        """Прогнозирование движения цены"""
        prediction = self.model.predict(X_input, verbose=0)[0][0]
        return prediction

    def display_info(self, prediction, current_price, bid_absorption, ask_absorption):
        """Визуализация рыночных данных и прогнозов"""
        clear_output(wait=True)
        prob_up = prediction * 100
        prob_down = (1 - prediction) * 100

        # Расчет волатильности для прогноза
        historical_volatility = self.calculate_volatility()
        expected_change = historical_volatility * 0.01 * (2 * prediction - 1)
        price_1m = current_price * (1 + expected_change)

        # Определение направления и уверенности
        direction = "рост" if prediction > 0.5 else "падение"
        confidence = max(prob_up, prob_down)

        # Причина прогноза
        if ask_absorption > bid_absorption:
            reason = f"Поглощение продаж ({ask_absorption:.2f} SOL) превышает поглощение покупок ({bid_absorption:.2f} SOL)"
        else:
            reason = f"Поглощение покупок ({bid_absorption:.2f} SOL) превышает поглощение продаж ({ask_absorption:.2f} SOL)"

        # Расчет диапазона для 5 минут на основе волатильности
        volatility_5m = historical_volatility * np.sqrt(5)
        lower_bound = current_price * (1 - volatility_5m * 0.01)
        upper_bound = current_price * (1 + volatility_5m * 0.01)

        # Вывод информации
        print(f"📊 Анализ {self.symbol} [{time.strftime('%H:%M:%S')}]")
        print(f"🏷 Текущая цена: {current_price:.4f}")
        print(f"📈 Заполнение буфера: {len(self.data_buffer)}/{self.BUFFER_SIZE}")
        print("\n🔍 Краткосрочный прогноз:")
        print(f"Прогноз через 1 мин: {price_1m:.4f} USDT ({direction})")
        print(f"Уверенность: {confidence:.1f}%")
        print(f"Причина: {reason}")
        print(f"Точность модели на исторических данных: {self.model_accuracy:.1f}%")
        print("\n📈 Среднесрочные прогнозы:")
        print(f"• Ожидаемый диапазон (+5 мин): {lower_bound:.4f} - {upper_bound:.4f}")
        print(f"• Ключевые уровни:")
        print(f"  - Поддержка: {current_price * 0.995:.4f}")
        print(f"  - Сопротивление: {current_price * 1.005:.4f}")
        print("\n📌 Сигналы ликвидности:")
        print(f"• Поглощение покупок: {bid_absorption:.2f} SOL")
        print(f"• Поглощение продаж: {ask_absorption:.2f} SOL")

    async def run_analysis(self):
        """Основной цикл анализа"""
        while True:
            asks, bids, current_price = await self.fetch_order_book()
            if asks and bids and current_price:
                processed_data = self.process_data(asks, bids, current_price)
                if processed_data is not None:
                    scaled_data = self.scaler.transform(processed_data[self.FEATURES])
                    self.data_sequence.append(scaled_data[0])
                    self.data_buffer.append(processed_data.iloc[0].to_dict())

                    # Начальное обучение на 200 значениях
                    if not self.initial_training_done and len(self.data_buffer) >= self.INITIAL_TRAIN_SIZE:
                        print("Выполняется начальное обучение модели...")
                        await self.retrain_model()
                        self.initial_training_done = True

                    # Прогнозирование
                    if len(self.data_sequence) >= self.SEQUENCE_LENGTH:
                        X_input = np.array(list(self.data_sequence)).reshape(1, self.SEQUENCE_LENGTH, -1)
                        prediction = self.predict_movement(X_input)
                        bid_absorption = processed_data['bid_absorption'].values[0]
                        ask_absorption = processed_data['ask_absorption'].values[0]
                        self.display_info(prediction, current_price, bid_absorption, ask_absorption)

                    # Переобучение после 1000 значений и 5 минут
                    if (self.initial_training_done and
                        len(self.data_buffer) >= self.BUFFER_SIZE and
                        time.time() - self.last_retrain_time > self.RETRAIN_INTERVAL):
                        print("Переобучение модели на полном буфере...")
                        await self.retrain_model()
                        self.last_retrain_time = time.time()

            self.driver.refresh()
            await asyncio.sleep(1)

# Запуск анализатора
if __name__ == "__main__":
    analyzer = MarketAnalyzer("SOL-USDT-SWAP")
    try:
        asyncio.run(analyzer.run_analysis())
    except KeyboardInterrupt:
        print("Анализ завершен пользователем.")

📊 Анализ SOL-USDT-SWAP [12:04:55]
🏷 Текущая цена: 133.1200
📈 Заполнение буфера: 65/1000

🔍 Краткосрочный прогноз:
Прогноз через 1 мин: 133.1226 USDT (рост)
Уверенность: 53.5%
Причина: Поглощение продаж (8938.50 SOL) превышает поглощение покупок (8062.09 SOL)
Точность модели на исторических данных: 0.0%

📈 Среднесрочные прогнозы:
• Ожидаемый диапазон (+5 мин): 133.0363 - 133.2037
• Ключевые уровни:
  - Поддержка: 132.4544
  - Сопротивление: 133.7856

📌 Сигналы ликвидности:
• Поглощение покупок: 8062.09 SOL
• Поглощение продаж: 8938.50 SOL
